In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.python.keras.layers import Conv1D, Concatenate, Dense, Flatten, GlobalAveragePooling1D, MaxPooling1D, Reshape
from tensorflow.python.keras.models import Model, Sequential

/Users/jaba/anaconda3/envs/jtrade/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
print(tf.__version__)
print(tf.keras.__version__)

1.7.1
2.1.4-tf


In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 8)

Load Images
==

In [20]:
import PIL
print(PIL.__version__)
from PIL import Image
from PIL import ImageOps

6.1.0


In [21]:
im = Image.open('./img/radius10/5/dots_005_00000.png')
print(im.format)
print(im.mode)
print(im.size)

# Convert to from RGB to grayscale
im = im.convert(mode='L')
print(im.mode)
print(im.size)

# Invert image so blobs are high and background is low
im = ImageOps.invert(im)

PNG
RGB
(128, 128)
L
(128, 128)


In [24]:
im.show()

In [23]:
# Convert to numpy array and convert values from 0 -> 255 to 0.0 -> 1.0
x = np.asarray(im) / 255.0
print(x)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
